In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-cardio-train/cardio_train.csv


In [2]:
file_path ='/kaggle/input/dataset-cardio-train/cardio_train.csv'
data=pd.read_csv(file_path,sep =';',index_col='id')

In [3]:
data.columns

Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol',
       'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')

In [4]:
data['age']=data['age'].map(lambda x : round(x/365,1))
data["BMI"]=data["weight"]/((data["height"]/100)**2)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI
id,,,,,,,,,,,,,
0,50.4,2,168,62.0,110,80,1,1,0,0,1,0,21.967120
1,55.4,1,156,85.0,140,90,3,1,0,0,1,1,34.927679
2,51.7,1,165,64.0,130,70,3,1,0,0,0,1,23.507805
3,48.3,2,169,82.0,150,100,1,1,0,0,1,1,28.710479
4,47.9,1,156,56.0,100,60,1,1,0,0,0,0,23.011177


In [5]:
y=data.cardio

In [6]:
feature_names = ['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol',
       'gluc', 'smoke', 'alco', 'active']
X= data[feature_names]
#X.describe()

#######
DATA=data.copy()
DATA = DATA.drop(["height","weight"],axis=1)
DATA

,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI
id,,,,,,,,,,,
0,50.4,2,110,80,1,1,0,0,1,0,21.967120
1,55.4,1,140,90,3,1,0,0,1,1,34.927679
2,51.7,1,130,70,3,1,0,0,0,1,23.507805
3,48.3,2,150,100,1,1,0,0,1,1,28.710479
4,47.9,1,100,60,1,1,0,0,0,0,23.011177
...,...,...,...,...,...,...,...,...,...,...,...
99993,52.7,2,120,80,1,1,1,0,1,0,26.927438
99995,61.9,1,140,90,2,2,0,0,1,1,50.472681
99996,52.2,2,180,90,3,1,0,1,0,1,31.353579


# Plotting

In [ ]:
import seaborn as sns

In [ ]:
import seaborn as sns
#sns.set()
#sns.scatterplot(x=data['cardio'], y=data['BMI'])
G = sns.PairGrid(data)
G.map(sns.scatterplot)

In [ ]:
GRAPH = sns.PairGrid(DATA)
GRAPH.map(sns.scatterplot)

In [ ]:
sns.heatmap(DATA.corr(), annot=True, fmt='.2f')

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(data['cardio'],data['age'])

In [ ]:
plt.scatter(data['cardio'],data['gender'])

# Test-train split

In [7]:
from sklearn.model_selection import train_test_split as TTS
from sklearn.metrics import accuracy_score

train_X, val_X, train_y, val_y = TTS(X,y,random_state=1)

# AUC Score

In [8]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

 # Decision Tree

In [9]:
from sklearn.tree import DecisionTreeRegressor as DTR

def decision_tree_regressor_predict_proba(X_train, y_train, X_test, **kwargs):
    """Trains DecisionTreeRegressor model and predicts probabilities of each y.

    Args:
        X_train: Training features.
        y_train: Training labels.
        X_test: New data to predict on.
        **kwargs: Other arguments passed to DecisionTreeRegressor.

    Returns:
        DataFrame with columns for record_id (row of X_test), y 
        (predicted value), and prob (of that y value).
        The sum of prob equals 1 for each record_id.
    """
    m = DecisionTreeRegressor(**kwargs).fit(X_train, y_train)

    node_ys = pd.DataFrame({'node_id': m.apply(X_train), 'y': y_train})

    node_ys['prob'] = 1 / node_ys.groupby(node_ys.node_id).transform('count')

    node_ys_dedup = node_ys.groupby(['node_id', 'y']).prob.sum().to_frame()\
        .reset_index()

    leaf = pd.DataFrame(m.decision_path(X_test).toarray()).apply(
        lambda x:x.to_numpy().nonzero()[0].max(), axis=1).to_frame(
            name='node_id')
    leaf['record_id'] = leaf.index

    return leaf.merge(node_ys_dedup, on='node_id').drop(
        'node_id', axis=1).sort_values(['record_id', 'y'])


In [10]:
DT_model=DTR(random_state=1)

DT_model.fit(train_X,train_y)

DT_predictions = DT_model.predict(val_X)


# pred_prob_DT = decision_tree_regressor_predict_proba(train_X,train_y,val_X,random_state=1, min_samples_leaf=5)

# fpr_DT, tpr_DT, thresh_DT = roc_curve(val_y, pred_prob_DT[:,1], pos_label=1)


In [11]:
from sklearn.metrics import mean_squared_error as MSE

In [12]:
DT_val_mse = MSE(val_y, DT_predictions)
print("MSE         : ",DT_val_mse)
print("Model score : ",DT_model.score(train_X,train_y))

MSE         :  0.36615755555555557
Model score :  0.9903945547968379


# Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression as LR
# solver='sag'
LR_model=LR(max_iter=1000)
LR_model.fit(train_X,train_y)

LR_predictions = LR_model.predict(val_X)
#print(LR_predictions[:10])
Accuracy_LR = accuracy_score(val_y, LR_predictions)
print("Logistic Regression Accuracy:", Accuracy_LR)

pred_prob_LR = LR_model.predict_proba(val_X)
fpr_LR, tpr_LR, thresh_LR = roc_curve(val_y, pred_prob_LR[:,1], pos_label=1)
auc_score_LR = roc_auc_score(val_y, pred_prob_LR[:,1])


Logistic Regression Accuracy: 0.7201714285714286


In [14]:
LR_val_mse = MSE(val_y, LR_predictions)
print("MSE         : ",LR_val_mse)
print("Model score : ",LR_model.score(train_X,train_y))
print("AUC Score   : ",auc_score_LR)

MSE         :  0.2798285714285714
Model score :  0.7226857142857143
AUC Score   :  0.787616928811908


# Support Vector Machines

In [ ]:
from sklearn.svm import SVC

SVM_model=SVC(probability=True)
SVM_model.fit(train_X,train_y)

SVM_predictions = SVM_model.predict(val_X)
print(SVM_predictions[:10])

Accuracy_SVM = accuracy_score(val_y, SVM_predictions)
print("Support Vector Machine Accuracy:", Accuracy_SVM)

pred_prob_SVM = SVM_model.predict_proba(val_X)
fpr_SVM, tpr_SVM, thresh_SVM = roc_curve(val_y, pred_prob_SVM[:,1], pos_label=1)
auc_score_SVM = roc_auc_score(val_y, pred_prob_SVM[:,1])

y.head(10)

In [ ]:
SVM_val_mse = MSE(val_y, SVM_predictions)
print("MSE         : ",SVM_val_mse)
print("Model score : ",SVM_model.score(train_X,train_y))

# Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier as RF

RF_model = RF()

RF_model.fit(train_X, train_y)

RF_predictions = RF_model.predict(val_X)

accuracy = accuracy_score(val_y, RF_predictions)
pred_prob_RF = RF_model.predict_proba(val_X)
fpr_RF, tpr_RF, thresh_RF = roc_curve(val_y, pred_prob_RF[:,1], pos_label=1)
auc_score_RF = roc_auc_score(val_y, pred_prob_RF[:,1])

y.head(10)
print("Random Forest Accuracy:", accuracy)

Random Forest Accuracy: 0.7152


In [16]:
RF_val_mae = MSE(val_y, RF_predictions)
print("MSE         : ",RF_val_mae)
print("Model score : ",RF_model.score(train_X,train_y))
print("AUC Score   : ",auc_score_RF)

MSE         :  0.2848
Model score :  0.9953714285714286
AUC Score   :  0.7740255921998174


# KNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier

KNN_model = KNeighborsClassifier()

KNN_model.fit(train_X, train_y)

KNN_predictions = KNN_model.predict(val_X)
pred_prob_KNN = KNN_model.predict_proba(val_X)
fpr_KNN, tpr_KNN, thresh_KNN = roc_curve(val_y, pred_prob_KNN[:,1], pos_label=1)
auc_score_KNN = roc_auc_score(val_y, pred_prob_KNN[:,1])

accuracy_KNN = accuracy_score(val_y, KNN_predictions)
print("KNN Accuracy:", accuracy_KNN)

KNN Accuracy: 0.6884


In [18]:
KNN_val_mae = MSE(val_y, KNN_predictions)
print("MSE         : ",KNN_val_mae)
print("Model score : ",KNN_model.score(train_X,train_y))
print("AUC Score   : ",auc_score_KNN)

MSE         :  0.3116
Model score :  0.7812
AUC Score   :  0.7347880662405869


# Scores
## MSE Scores
> 1. Decision Tree       :  0.36615755555555557
> 2. Logistic Regression :  0.2798285714285714
> 3. Random Forest       : 0.2848
> 4. KNN                 : 0.3116

## In-built Model Score_Calulator
> 1. Decision Tree       :  0.9903945547968379
> 2. Logistic Regression :  0.7226857142857143
> 3. Random Forest       :  0.9953714285714286
> 4. KNN                 :  0.7812

## AUC Scores
> 1. Logistic Regression :  0.787616928811908
> 2. Random Forest       :  0.7740255921998174
> 3. KNN                 :  0.7347880662405869